In [1]:
import numpy as np
import csv
import scipy
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset



tlist = scipy.io.loadmat('Heston' + '_tlist.mat')
tlist = tlist['tlist'] # note that they are 2D array 
tlist = tlist.reshape((-1))
# Klist = scipy.io.loadmat('msft_Klist.mat')
Klist = scipy.io.loadmat('Heston' + '_Klist.mat')
Klist = Klist['Klist'] # note that they are 2D array
Klist = Klist.reshape((-1))
totalvarsurface = scipy.io.loadmat('HestonIVSgrid_NI91529.mat')
# totalvarsurface = scipy.io.loadmat('HestonIVSgrid_NI.mat')
# totalvarsurface = scipy.io.loadmat('HestonIVSgrid_NI87527.mat')

# totalvarsurface = totalvarsurface['totalvarsurface']
totalvarsurface = totalvarsurface['HestonIVS2D']
IVStrain, IVStest = train_test_split(totalvarsurface, test_size = 0.2, random_state = 42)

In [2]:
image_size = 8
i_d = int(np.ceil(image_size/4))
mask = np.full((image_size,image_size), True)
indexlist = [(i_d, i_d), (i_d, -i_d-1), (-i_d-1, i_d), (-i_d-1,-i_d-1)]
for x in indexlist:
    mask[x] = False

In [3]:
IVStest.shape

(17506, 16, 16)

In [4]:
# Let's compare to simple interpolation. neighbor IV mid point value
# testsize = IVStest.shape[0]
# testsize = 100
testsize = 100
errorlist = []
maxerrorlist = []
for x in indexlist:
    allerrorlist = []
    x1 = x[0]
    x2 = x[1]
    newid1 = (x1-1, x2)
    newid2 = (x1+1, x2)
    for i in range(testsize):
        testsurf = IVStest[i]
        y_true = testsurf[x]
        y_test = (testsurf[newid1] + testsurf[newid2])/2
        error = np.abs(y_true -y_test) / np.abs(y_true )
        allerrorlist.append(error)
    allerror = np.array(allerrorlist)
    avgerror = np.mean(allerror)
    maxerror = np.amax(allerror)
    errorlist.append(avgerror)
    maxerrorlist.append(maxerror)
         
        

In [5]:
errorlistnp = np.array(errorlist)
print(errorlistnp * 100)
maxerrorlistnp = np.array(maxerrorlist)
maxerrorlistnp * 100

[0.01477991 0.02122595 0.00742994 0.00939677]


array([0.13008515, 0.3131735 , 0.03537981, 0.05485178])

In [102]:
l = np.mean([1,2,3,4,5])
np.mean(l)


3.0

In [17]:
for x in indexlist:
    print(x)
    y = x
    y

(2, 2)
(2, -3)
(-3, 2)
(-3, -3)


# Some other randomness testing

In [44]:
IVStrain, IVStest = train_test_split(totalvarsurface, test_size = 0.2, random_state = 42)


In [45]:
IVStrain1, IVStest1 = train_test_split(totalvarsurface, test_size = 0.2, random_state = 42)


In [50]:
IVStest.shape

(18306, 8, 8)

In [51]:
18306 * 64

1171584

In [47]:
np.sum(IVStest == IVStest1)


1171584

In [54]:
from datasets.Dset import * 

In [90]:
torch.manual_seed(0)
dataset = Dset(IVStest)
dataloader = DataLoader(dataset, batch_size=128,shuffle=True, drop_last=True)
refer_image = next(iter(dataloader))

In [91]:
refer_image.shape

torch.Size([128, 8, 8])

In [92]:
torch.manual_seed(1)
dataset = Dset(IVStest)
dataloader = DataLoader(dataset, batch_size=128,shuffle=True, drop_last=True)
refer_image1 = next(iter(dataloader))


In [93]:
np.sum(np.array(refer_image == refer_image1))

0

In [94]:
128 * 64

8192

# plotting

In [107]:
import os
# Let's compare to simple interpolation. neighbor IV mid point value
# testsize = IVStest.shape[0]
# testsize = 100
testsize = 100
errorlist = []
maxerrorlist = []
for x in indexlist:
    allerrorlist = []
    x1 = x[0]
    x2 = x[1]
    newid1 = (x1-1, x2)
    newid2 = (x1+1, x2)
    for i in range(testsize):
        testsurf = IVStest[i]
        y_true = testsurf[x]
        y_test = (testsurf[newid1] + testsurf[newid2])/2
        error = np.abs(y_true -y_test) / np.abs(y_true )
        allerrorlist.append(error)
    allerror = np.array(allerrorlist)
    avgerror = np.mean(allerror)
    maxerror = np.amax(allerror)
    errorlist.append(avgerror)
    maxerrorlist.append(maxerror)

for i in range(testsize):  
    testsurf = IVStest[i]
    predsurf = testsurf.copy()
    for x in indexlist:
        x1 = x[0]
        x2 = x[1]
        newid1 = (x1-1, x2)
        newid2 = (x1+1, x2)
        y_test = (testsurf[newid1] + testsurf[newid2])/2
        predsurf[x1,x2] = y_test
    savepath = os.path.join('imagefolder', 'ivs_image_inpainting{}.png'.format(i))
    IVS_visualize(predsurf, Klist, tlist, savepath=savepath, plotname = str(i) + "baseline")



         
        